## Installation and set libraries

In [1]:
# !pip install nba_api pandas
from time import sleep
import pandas as pd

from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog, playbyplayv2, shotchartdetail



#### Team ID lakers and Match IDs

In [2]:
def get_team_id(team_full_name="Los Angeles Lakers"):
    t = teams.find_teams_by_full_name(team_full_name)
    assert len(t) == 1, f"Trovati {len(t)} team"
    return t[0]["id"]

LAL_ID = get_team_id()
LAL_ID


1610612747

In [24]:
from nba_api.stats.endpoints import leaguegamefinder

def get_team_game_ids_finder(team_id: int, season: str):
    gf = leaguegamefinder.LeagueGameFinder(
        team_id_nullable=team_id,
        season_nullable=season           # es. "2024-25"
    )
    games = gf.get_data_frames()[0]
    games = games[games["GAME_ID"].astype(str).str.startswith("002")] # if you want only the regular season
    return games["GAME_ID"].dropna().astype(str).unique().tolist()

game_ids = get_team_game_ids_finder(LAL_ID, "2024-25")
len(game_ids)

82

## Fetching data play_by_play

- fetch_pbp: 1 game
- fetch_pbp_season: all season

In [25]:
def fetch_pbp(game_id: str) -> pd.DataFrame:
    pbp = playbyplayv2.PlayByPlayV2(game_id=game_id)
    df = pbp.get_data_frames()[0]
    df["GAME_ID"] = game_id
    # colonne chiave: EVENTMSGTYPE, EVENTMSGACTIONTYPE, HOMEDESCRIPTION, NEUTRALDESCRIPTION, VISITORDESCRIPTION, PCTIMESTRING, PERIOD
    return df


In [ ]:
def fetch_pbp_season(game_ids, sleep_sec=0.6) -> pd.DataFrame:
    frames = []
    for gid in game_ids:
        frames.append(fetch_pbp(gid))
        sleep(sleep_sec)  # cortesia verso l'endpoint
    return pd.concat(frames, ignore_index=True)

pbp_all = fetch_pbp_season(game_ids)
pbp_all.shape


      GAME_ID  EVENTNUM  EVENTMSGTYPE  EVENTMSGACTIONTYPE  PERIOD  \
0  0022401199         2            12                   0       1   
1  0022401199         4            10                   0       1   
2  0022401199         7             2                   6       1   
3  0022401199         8             4                   0       1   
4  0022401199        10             2                   1       1   

  WCTIMESTRING PCTIMESTRING                           HOMEDESCRIPTION  \
0      3:44 PM        12:00                                      None   
1      3:44 PM        12:00  Jump Ball Clingan vs. Len: Tip to Camara   
2      3:44 PM        11:45            MISS Thybulle 6' Driving Layup   
3      3:44 PM        11:43                                      None   
4      3:45 PM        11:25                                      None   

                  NEUTRALDESCRIPTION             VISITORDESCRIPTION  ...  \
0  Start of 1st Period (3:44 PM EST)                           None  .

In [31]:
pbp_all[pbp_all["EVENTMSGACTIONTYPE"] == 2]

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
20,0022401199,29,6,2,1,3:47 PM,9:16,None,None,Morris S.FOUL (P1.T1) (E.Scott),...,Trail Blazers,POR,1,0,None,NaN,None,None,None,1
30,0022401199,44,6,2,1,3:52 PM,8:37,None,None,Len S.FOUL (P1.T2) (N.Buchert),...,Trail Blazers,POR,1,0,None,NaN,None,None,None,1
38,0022401199,54,6,2,1,3:53 PM,7:44,None,None,Len S.FOUL (P2.T4) (D.Guthrie),...,Trail Blazers,POR,1,0,None,NaN,None,None,None,1
50,0022401199,68,6,2,1,3:55 PM,6:33,Clingan S.FOUL (P1.T2) (N.Buchert),None,None,...,Lakers,LAL,1,0,None,NaN,None,None,None,1
58,0022401199,80,1,2,1,3:57 PM,6:08,None,None,Milton 27' 3PT Running Jump Shot (3 PTS) (Morr...,...,Lakers,LAL,0,0,None,NaN,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37638,0022400062,533,1,2,4,12:01 AM,10:38,None,None,Reid 24' 3PT Running Jump Shot (12 PTS) (McDan...,...,Timberwolves,MIN,0,0,None,NaN,None,None,None,1
37669,0022400062,577,6,2,4,12:13 AM,7:09,None,None,Reid S.FOUL (P2.PN) (J.Williams),...,Lakers,LAL,1,0,None,NaN,None,None,None,1
37687,0022400062,601,6,2,4,12:17 AM,5:31,None,None,Reid S.FOUL (P3.PN) (N.Buchert),...,Lakers,LAL,1,0,None,NaN,None,None,None,1
37716,0022400062,643,6,2,4,12:25 AM,1:56,Reaves S.FOUL (P4.T4) (N.Buchert),None,None,...,Timberwolves,MIN,1,0,None,NaN,None,None,None,1


## Possible Explanations

#### EVENTMSGTYPE

1 - Make 2 - Miss 3 - Free Throw 4 - Rebound 5 - out of bounds / Turnover / Steal 6 - Personal Foul 7 - Violation 8 - Substitution 9 - Timeout 10 - Jumpball 12 - Start Q1? 13 - Start Q2?

#### EVENTMSGACTIONTYPE

1 - Jumpshot 2 - Lost ball Turnover 3 - ? 4 - Traveling Turnover / Off Foul 5 - Layup 7 - Dunk 10 - Free throw 1-1 11 - Free throw 1-2 12 - Free throw 2-2 40 - out of bounds 41 - Block/Steal 42 - Driving Layup 50 - Running Dunk 52 - Alley Oop Dunk 55 - Hook Shot 57 - Driving Hook Shot 58 - Turnaround hook shot 66 - Jump Bank Shot 71 - Finger Roll Layup 72 - Putback Layup 108 - Cutting Dunk Shot

In [ ]:
sc = shotchartdetail.ShotChartDetail(
    team_id=LAL_ID,
    player_id=0,                 # 0 = tutti i giocatori della squadra
    season_type_all_star="Regular Season",
    season_nullable="2024-25",
    context_measure_simple="FGA"    
)

shots = sc.get_data_frames()[0]
print(shots.shape)
print(shots.columns)
shots.head(30)
# unita di misura decimo di piede

(7007, 24)
Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')


755